## 文件读写
读写文件是最常见的IO操作。Python内置了读写文件的函数，用法和C是兼容的。

读写文件前，我们先必须了解一下，在磁盘上读写文件的功能都是由操作系统提供的，现代操作系统不允许普通的程序直接操作磁盘，所以，读写文件就是请求操作系统打开一个文件对象（通常称为文件描述符），然后，通过操作系统提供的接口从这个文件对象中读取数据（读文件），或者把数据写入这个文件对象（写文件）。

### 读文件
要以读文件的模式打开一个文件对象，使用Python内置的open()函数，传入文件名和标示符：

In [8]:
try:
    f = open('./test1.html', 'r')
    dir(f)
    f.read()
except BaseException as e:
    print(e)
    print(f)
finally:
    f.close()
    

[Errno 2] No such file or directory: './test1.html'
<_io.TextIOWrapper name='./test.html' mode='r' encoding='cp936'>


In [9]:
try:
    f = open('./log.txxt','r')
    f.__dir__()
    str1 = f.read()
    print(str1)
except Exception as e:
    print(e)
finally:
    f.close()

[Errno 2] No such file or directory: './log.txxt'


In [4]:
f = open('./test.txt', 'r')
f.read()

'helloWorld'

### 调用close()方法关闭文件。
文件使用完毕后必须关闭，因为文件对象会占用操作系统的资源，并且操作系统同一时间能打开的文件数量也是有限的

In [5]:
f.close()

由于文件读写时都有可能产生IOError，一旦出错，后面的f.close()就不会调用。所以，为了保证无论是否出错都能正确地关闭文件，我们可以使用try ... finally来实现：

In [7]:
try:
    f = open('./test.txt', 'r')
    print(f.read())
finally:
    if f:
        f.close()

helloWorld


### with语句自动调用close()方法
但是每次都这么写实在太繁琐，所以，Python引入了with语句来自动帮我们调用close()方法：

In [11]:
with open('./test.txt', 'r') as f:
    print(f.read())
    

helloWorld


In [12]:
with open('./test.html','r') as f:
    while True:
        a = f.readline()
        print(a)
        if not a:
            break
            
        
#     print(f.read())
# f.close()

<h1>你好!<h1>



In [14]:
with open('./test.html','r') as f:
    print(f.readlines())
    
dir(f)

['<h1>你好!<h1>']


['_CHUNK_SIZE',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_checkClosed',
 '_checkReadable',
 '_checkSeekable',
 '_checkWritable',
 '_finalizing',
 'buffer',
 'close',
 'closed',
 'detach',
 'encoding',
 'errors',
 'fileno',
 'flush',
 'isatty',
 'line_buffering',
 'mode',
 'name',
 'newlines',
 'read',
 'readable',
 'readline',
 'readlines',
 'seek',
 'seekable',
 'tell',
 'truncate',
 'writable',
 'write',
 'writelines']

调用read()会一次性读取文件的全部内容，如果文件有10G，内存就爆了，所以，
* 要保险起见，可以反复调用read(size)方法，每次最多读取size个字节的内容。
* 调用readline()可以每次读取一行内容
* 调用readlines()一次读取所有内容并按行返回list。

因此，要根据需要决定怎么调用。

如果文件很小，read()一次性读取最方便；如果不能确定文件大小，反复调用read(size)比较保险；如果是配置文件，调用readlines()最方便：

In [13]:
f = open('./test.txt', 'r')
for line in f.readlines():
    print(line.strip()) # 把末尾的'\n'删掉

helloWorld


In [13]:
with open('./log.txt','r') as f:
#     print(f.readlines())
    for item in range(3):
        print(f.readline())
    

2018-05-11 14:35:10,593 - log.py[line:9] - INFO: this is a loggging info message

2018-05-11 14:35:10,594 - log.py[line:10] - DEBUG: this is a loggging debug message

2018-05-11 14:35:10,594 - log.py[line:11] - WARNING: this is loggging a warning message



### file-like Object（类文件对象）
像open()函数返回的这种有个read()方法的对象，在Python中统称为file-like Object。除了file外，还可以是内存的字节流，网络流，自定义流等等。file-like Object不要求从特定类继承，只要写个read()方法就行。

StringIO就是在内存中创建的file-like Object，常用作临时缓冲。

In [ ]:
# class Student:
#     def read(self):
#         return '123456'
#     def close(self):
#         return '文件关闭'
    


### 二进制文件
前面讲的默认都是读取文本文件，并且是UTF-8编码的文本文件。要读取二进制文件，比如图片、视频等等，用'rb'模式打开文件即可：

In [15]:
f = open('./test.html', 'rb')
f.read()

b'<h1>\xc4\xe3\xba\xc3!<h1>'

### 字符编码
要读取非UTF-8编码的文本文件，需要给open()函数传入encoding参数，例如，读取GBK编码的文件：

In [23]:
f = open('./test.html', 'r', encoding='utf-8')
f.read()

'\ufeff<h1>你好!<h1>'

遇到有些编码不规范的文件，你可能会遇到UnicodeDecodeError，因为在文本文件中可能夹杂了一些非法编码的字符。遇到这种情况，open()函数还接收一个errors参数，表示如果遇到编码错误后如何处理。最简单的方式是直接忽略：

In [26]:
f = open('./test.html', 'r', encoding='utf-8', errors='ignore')
f.read()

'\ufeff<h1>你好!<h1>'

### 写文件
写文件和读文件是一样的，唯一区别是调用open()函数时，传入标识符'w'或者'wb'表示写文本文件或写二进制文件：



In [30]:
with open('./stduents.txt','a') as f:
    f.write('\n小黑 男 三年二班')

# with open('./stduents.txt','w') as f:
#     f.write('小红 女 三年二班')    

In [47]:
with open('./stduents.txt','r') as f:
    students = f.readlines()
students = list(students)
dir(students[1])
for item in students:
    student = item.split(' ')
    print('学生名字：%s'%student[0])
    print('学生性别：%s'%student[1])
    print('学生年级：%s'%student[2])
    


学生名字：小红
学生性别：女
学生年级：三年二班

学生名字：小李
学生性别：男
学生年级：三年二班

学生名字：小黑
学生性别：男
学生年级：三年二班


In [43]:
help(str.split)

Help on method_descriptor:

strip(...)
    S.strip([chars]) -> str
    
    Return a copy of the string S with leading and trailing
    whitespace removed.
    If chars is given and not None, remove characters in chars instead.



In [1]:
# f = open('./test.txt', 'w')
# f.write('Hello, world!')
# f.close()

In [18]:
with open('./log.txt','a') as f:
    print(f)
    print(f.__dir__())
    f.write('欢迎你xxx')

<_io.TextIOWrapper name='./log.txt' mode='a' encoding='cp936'>
['mode', '__repr__', '__next__', '__init__', '__new__', 'detach', 'write', 'read', 'readline', 'flush', 'close', 'fileno', 'seekable', 'readable', 'writable', 'isatty', '__getstate__', 'seek', 'tell', 'truncate', 'encoding', 'buffer', 'line_buffering', '_finalizing', 'name', 'closed', 'newlines', 'errors', '_CHUNK_SIZE', '__doc__', '__iter__', '__del__', '_checkClosed', '_checkSeekable', '_checkReadable', '_checkWritable', '__enter__', '__exit__', 'readlines', 'writelines', '__dict__', '__hash__', '__str__', '__getattribute__', '__setattr__', '__delattr__', '__lt__', '__le__', '__eq__', '__ne__', '__gt__', '__ge__', '__reduce_ex__', '__reduce__', '__subclasshook__', '__init_subclass__', '__format__', '__sizeof__', '__dir__', '__class__']


你可以反复调用write()来写入文件，但是务必要调用f.close()来关闭文件。当我们写文件时，操作系统往往不会立刻把数据写入磁盘，而是放到内存缓存起来，空闲的时候再慢慢写入。只有调用close()方法时，操作系统才保证把没有写入的数据全部写入磁盘。忘记调用close()的后果是数据可能只写了一部分到磁盘，剩下的丢失了。所以，还是用with语句来得保险：



In [34]:
with open('./test.html', 'w') as f:
    f.write('<h1>你好!<h1>')

要写入特定编码的文本文件，请给open()函数传入encoding参数，将字符串自动转换成指定编码。

细心的童鞋会发现，以'w'模式写入文件时，如果文件已存在，会直接覆盖（相当于删掉后新写入一个文件）。如果我们希望追加到文件末尾怎么办？可以传入'a'以追加（append）模式写入。

In [31]:
with open('./cc.txt','w') as f:
    f.write('nihao')
    f.write('666')

In [35]:
help(open)

Help on built-in function open in module io:

open(file, mode='r', buffering=-1, encoding=None, errors=None, newline=None, closefd=True, opener=None)
    Open file and return a stream.  Raise IOError upon failure.
    
    file is either a text or byte string giving the name (and the path
    if the file isn't in the current working directory) of the file to
    be opened or an integer file descriptor of the file to be
    wrapped. (If a file descriptor is given, it is closed when the
    returned I/O object is closed, unless closefd is set to False.)
    
    mode is an optional string that specifies the mode in which the file
    is opened. It defaults to 'r' which means open for reading in text
    mode.  Other common values are 'w' for writing (truncating the file if
    it already exists), 'x' for creating and writing to a new file, and
    'a' for appending (which on some Unix systems, means that all writes
    append to the end of the file regardless of the current seek position

In [36]:
'你好' in '我你好啊'

True